In [43]:
from test_ifidf import get_tf_title
import os
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split
from gensim.models import word2vec
import pickle
import logging
import numpy

In [46]:
train_file = os.path.abspath('./liar_dataset/train.tsv')
test_file = os.path.abspath('./liar_dataset/test.tsv')
train_infos, test_infos = get_tf_title(train_file), get_tf_title(test_file)
infos = [train_infos[0] + test_infos[0], train_infos[1] + test_infos[1]]

In [47]:
with open('quotes.txt', 'w') as f:
    quotes = infos[1]
    for quote in quotes:
        f.write(quote + "\n")

In [48]:
sentences = word2vec.Text8Corpus('quotes.txt')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = word2vec.Word2Vec(sentences, size=500, min_count=1, window=15)
model.save("./wiki.model")

2018-07-18 15:43:53,664 : INFO : collecting all words and their counts
2018-07-18 15:43:53,666 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-07-18 15:43:53,724 : INFO : collected 23409 word types from a corpus of 206936 raw words and 21 sentences
2018-07-18 15:43:53,726 : INFO : Loading a fresh vocabulary
2018-07-18 15:43:53,815 : INFO : min_count=1 retains 23409 unique words (100% of original 23409, drops 0)
2018-07-18 15:43:53,816 : INFO : min_count=1 leaves 206936 word corpus (100% of original 206936, drops 0)
2018-07-18 15:43:53,886 : INFO : deleting the raw counts dictionary of 23409 items
2018-07-18 15:43:53,887 : INFO : sample=0.001 downsamples 38 most-common words
2018-07-18 15:43:53,887 : INFO : downsampling leaves estimated 170977 word corpus (82.6% of prior 206936)
2018-07-18 15:43:53,936 : INFO : estimated required memory for 23409 words and 500 dimensions: 105340500 bytes
2018-07-18 15:43:53,937 : INFO : resetting layer weights
2018-07-18 

In [49]:
# https://medium.com/eureka-engineering/pairs%E3%81%AE%E3%82%B3%E3%83%9F%E3%83%A5%E3%83%8B%E3%83%86%E3%82%A3%E3%82%92word2vec%E3%81%A8svm%E3%81%A7%E5%88%86%E9%A1%9E%E3%81%97%E3%81%A6%E3%81%BF%E3%81%9F-48f4099f0ffc
def text_to_vec(words, model):
    word_vecs = []
    for word in words:
        try:
            word_vecs.append(model[word])
        except:
            pass
    if len(word_vecs) == 0:
        return None
    text_vec = numpy.zeros(word_vecs[0].shape, dtype = word_vecs[0].dtype)
    for word_vec in word_vecs:
        text_vec = text_vec + word_vec
    return text_vec

In [50]:
def normalize(vec):
    return vec / numpy.linalg.norm(vec)

In [51]:
with open('quotes.txt', 'w') as f:
    quotes = infos[1]
    features = []
    for quote in quotes:
        quote_words = quote.rstrip().split(' ')
        quote_name = ''.join(quote_words).replace("/", "")[:10]
        quote_vec = text_to_vec(quote_words, model)
        quote_vec = normalize(quote_vec)
        numpy.savetxt('./vec/' + quote_name, quote_vec)
        features.append(quote_vec)

/mnt/c/Users/yuta0/opensuse/.anyenv/envs/pyenv/versions/miniconda3-4.3.30/envs/ML_practice/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [52]:
labels = infos[0]
# トレーニングデータ:テストデータ を 9:1 に分割
data_train, data_test, label_train, label_test = train_test_split(features, labels, test_size=0.1, random_state=1)

In [53]:
%%time
# トレーニングデータから分類器を作成 (Linear SVM)
estimator = LinearSVC(C=1.0)
estimator.fit(data_train, label_train)
# テストデータを分類器に入れる
label_predict = estimator.predict(data_test)
# Fscore
print(f1_score(label_test, label_predict))

0.82818043588444
CPU times: user 922 ms, sys: 125 ms, total: 1.05 s
Wall time: 1.09 s
